## Sixth Session (Related to the Course Project)

---------------

## Graph Classification with [Deep Graph Library (DGL)](https://docs.dgl.ai/index.html) for the graduate course "[Graph Machine learning](https://github.com/zahta/graph_ml)"

### Dataset: bbbp

##### by [Zahra Taheri](https://github.com/zahta), 06 June 2023

---------------

### This Tutorial Is Prepared Based on the Following References

- [FunQG: Molecular Representation Learning via Quotient Graphs](https://pubs.acs.org/doi/10.1021/acs.jcim.3c00445)
- [Supporting Information of FunQG](https://pubs.acs.org/doi/suppl/10.1021/acs.jcim.3c00445/suppl_file/ci3c00445_si_001.pdf)
- [GitHub Repository of FunQG](https://github.com/hhaji/funqg)

In [1]:
pip install  dgl -f https://data.dgl.ai/wheels/repo.html

Looking in links: https://data.dgl.ai/wheels/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 68.6 MB/s eta 0:00:00


In [2]:
pip install  dglgo -f https://data.dgl.ai/wheels-test/repo.html

Looking in links: https://data.dgl.ai/wheels-test/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 95.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 60.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.7/572.7 kB 59.8 MB/s eta 0:00:00
  Created whee

In [3]:
%matplotlib inline
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import numpy as np
import networkx as nx
import torch
import torch.nn as nn
import dgl.function as fn
import torch.nn.functional as F
import shutil
from torch.utils.data import DataLoader
import cloudpickle
from dgl.nn import GraphConv
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from dgl.nn import GraphConv
from dgl.nn import GINConv
from dgl.nn import SAGEConv
from dgl.nn import GATConv

#### Set Path

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Define the path to the current directory where the ZIP file is located.
current_dir = "/content/drive/MyDrive/graph_data1.zip"

# Create the path to the directory where model checkpoints will be saved.
checkpoint_path = current_dir + "save_models/model_checkpoints/" + "checkpoint"
os.makedirs(checkpoint_path, exist_ok=True)

# Define the path to the directory where the best model will be saved.
best_model_path = current_dir + "save_models/best_model/"

# Create a temporary folder path for data manipulation.
folder_data_temp = current_dir + "data_temp/"

# Remove the temporary folder if it exists, ignoring any errors if it does not.
shutil.rmtree(folder_data_temp, ignore_errors=True)

# Define the path to save the unpacked files from the ZIP archive.
path_save = current_dir

# Unpack the contents of the ZIP archive to the temporary folder.
shutil.unpack_archive(path_save, folder_data_temp)


#### Custom PyTorch Datasets

In [6]:
""" Classification Dataset """

class DGLDatasetClass(torch.utils.data.Dataset):
    def __init__(self, address):
        # Initialize the dataset with the given address
        self.address = address + ".bin"
        self.list_graphs, train_labels_masks_globals = dgl.load_graphs(self.address)
        num_graphs = len(self.list_graphs)

        # Extract labels, masks, and globals from train_labels_masks_globals
        self.labels = train_labels_masks_globals["labels"].view(num_graphs, -1)
        self.masks = train_labels_masks_globals["masks"].view(num_graphs, -1)
        self.globals = train_labels_masks_globals["globals"].view(num_graphs, -1)

    def __len__(self):
        # Return the length of the dataset
        return len(self.list_graphs)

    def __getitem__(self, idx):
        # Retrieve the item at the given index
        return self.list_graphs[idx], self.labels[idx], self.masks[idx], self.globals[idx]


#### Defining Train, Validation, and Test Set

In [7]:
# Define the path for temporary data
path_data_temp = folder_data_temp + "scaffold" + "_" + str(0)

# Create instances of DGLDatasetClass for the train, validation, and test sets
train_set = DGLDatasetClass(address=path_data_temp + "_train")
val_set = DGLDatasetClass(address=path_data_temp + "_val")
test_set = DGLDatasetClass(address=path_data_temp + "_test")

# Print the lengths of the train, validation, and test sets
print(len(train_set), len(val_set), len(test_set))

1631 203 205


#### Data Loader

In [9]:
# Define a collate function to process a batch of data samples.
def collate(batch):
    # Extract the graphs from the batch and create a batched graph using dgl.batch.
    graphs = [e[0] for e in batch]
    g = dgl.batch(graphs)

    # Extract the labels from the batch and stack them into a tensor.
    labels = [e[1] for e in batch]
    labels = torch.stack(labels, 0)

    # Extract the masks from the batch and stack them into a tensor.
    masks = [e[2] for e in batch]
    masks = torch.stack(masks, 0)

    # Extract the global features from the batch and stack them into a tensor.
    globals = [e[3] for e in batch]
    globals = torch.stack(globals, 0)

    # Return the batched graph, labels, masks, and globals.
    return g, labels, masks, globals


# Define a loader function to create data loaders for the training, validation, and test sets.
def loader(batch_size=64):
    # Create a data loader for the training set.
    train_dataloader = DataLoader(train_set,
                                  batch_size=batch_size,
                                  collate_fn=collate,
                                  drop_last=False,
                                  shuffle=True,
                                  num_workers=1)

    # Create a data loader for the validation set.
    val_dataloader = DataLoader(val_set,
                                batch_size=batch_size,
                                collate_fn=collate,
                                drop_last=False,
                                shuffle=False,
                                num_workers=1)

    # Create a data loader for the test set.
    test_dataloader = DataLoader(test_set,
                                 batch_size=batch_size,
                                 collate_fn=collate,
                                 drop_last=False,
                                 shuffle=False,
                                 num_workers=1) #The number of worker threads to use for loading the data.

    # Return the data loaders for training, validation, and test sets.
    return train_dataloader, val_dataloader, test_dataloader


In [10]:
# Create data loaders for the training, validation, and test sets with a batch size of 64.
train_dataloader, val_dataloader, test_dataloader = loader(batch_size=64)

#### Defining A GNN

##### Some Variables

In [11]:
#Bace dataset has 1 task. Some other datasets may have some more number of tasks, e.g., tox21 has 12 tasks.
num_tasks = 1

# Size of global feature of each graph
global_size = 200

# Number of epochs to train the model
num_epochs = 100

# Number of steps to wait if the model performance on the validation set does not improve
patience = 10

#Configurations to instantiate the model
config = {"node_feature_size":127, "edge_feature_size":12, "hidden_size":100}


#GCN 2Layer

In [12]:
#Define a GNN (Graph Neural Network) class as a subclass of nn.Module(This GNN model can be used for graph classification tasks on molecular graphs. 
# The forward pass takes a DGL graph object and the global features as input and computes the node representations for each task)
class GNN(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        # Create the first GraphConv layer with input size equal to the node feature size and output size equal to the hidden size.
        self.conv1 = GraphConv(self.node_feature_size, self.hidden_size, allow_zero_in_degree = True)

        # Create the second GraphConv layer with input size equal to the hidden size and output size equal to the number of tasks
        self.conv2 = GraphConv(self.hidden_size, self.num_tasks, allow_zero_in_degree = True)

    # def forward(self, g, in_feat):
def forward(self, mol_dgl_graph, globals):
    # Reduce the node feature dimensionality
    mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
    # Reduce the edge feature dimensionality
    mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]
    # Perform convolution operation on the graph with the node features
    h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
    # Apply ReLU activation function
    h = F.relu(h)
    # Perform another convolution operation on the graph with the updated features
    h = self.conv2(mol_dgl_graph, h)
    # Assign the updated features to the node data
    mol_dgl_graph.ndata["h"] = h
    # Return the mean of the node features across the graph
    return dgl.mean_nodes(mol_dgl_graph, "h")


#### Function to Compute Score of the Model

In [13]:
from sklearn.metrics import roc_auc_score

def compute_score(model, data_loader, val_size, num_tasks):
    # Set the model to evaluation mode
    model.eval()

    # Define the metric as roc_auc_score
    metric = roc_auc_score

    with torch.no_grad():
        # Initialize empty tensors for storing predictions, labels, and masks
        prediction_all = torch.empty(0)
        labels_all = torch.empty(0)
        masks_all = torch.empty(0)

        # Iterate over the data_loader
        for i, (mol_dgl_graph, labels, masks, globals) in enumerate(data_loader):
            # Compute predictions from the model
            prediction = model(mol_dgl_graph, globals)
            prediction = torch.sigmoid(prediction)

            # Concatenate predictions, labels, and masks to the respective tensors
            prediction_all = torch.cat((prediction_all, prediction), 0)
            labels_all = torch.cat((labels_all, labels), 0)
            masks_all = torch.cat((masks_all, masks), 0)

        # Initialize average score tensor
        average = torch.tensor([0.])

        # Compute the metric for each task
        for i in range(num_tasks):
            a1 = prediction_all[:, i][masks_all[:, i] == 1]
            a2 = labels_all[:, i][masks_all[:, i] == 1]

            try:
                # Calculate the metric score
                t = metric(a2.int().cpu(), a1.cpu()).item()
            except ValueError:
                # Handle the case where the metric calculation throws a ValueError
                t = 0

            average += t

    # Return the average score divided by the number of tasks
    return average.item() / num_tasks


#### Loss Function

In [14]:
# Define the loss function
def loss_func(output, label, mask, num_tasks):
    # Create a tensor of ones as positive weights for BCEWithLogitsLoss
    pos_weight = torch.ones((1, num_tasks))
    pos_weight

    # Define the criterion as BCEWithLogitsLoss with no reduction and positive weights
    criterion = torch.nn.BCEWithLogitsLoss(reduction='none', pos_weight=pos_weight)

    # Compute the element-wise loss by applying the mask to the criterion output
    loss = mask * criterion(output, label)

    # Compute the average loss by summing the masked loss values and dividing by the sum of the mask values
    loss = loss.sum() / mask.sum()

    # Return the computed loss
    return loss



#### Training and Evaluation

##### Training Function

In [15]:
# Define a function to train a single epoch using the given training data loader, model, and optimizer.
def train_epoch(train_dataloader, model, optimizer):
    # Initialize the epoch train loss and iterations.
    epoch_train_loss = 0
    iterations = 0

    # Set the model to train mode.
    model.train()

    # Iterate over the training data loader.
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(train_dataloader):
        # Make predictions using the model.
        prediction = model(mol_dgl_graph, globals)

        # Compute the training loss using the loss function.
        loss_train = loss_func(prediction, labels, masks, num_tasks)

        # Zero the gradients of the model parameters.
        optimizer.zero_grad(set_to_none=True)

        # Perform backpropagation to compute gradients.
        loss_train.backward()

        # Update the model parameters using the optimizer.
        optimizer.step()

        # Accumulate the training loss.
        epoch_train_loss += loss_train.detach().item()

        # Increment the iterations count.
        iterations += 1

    # Compute the average epoch train loss.
    epoch_train_loss /= iterations

    # Return the average epoch train loss.
    return epoch_train_loss


In [16]:
# Define a function to train and evaluate the model.
def train_evaluate():
    # Create a new instance of the GNN model with the given configuration.
    model = GNN(config, global_size, num_tasks)

    # Create an Adam optimizer for training the model.
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

    # Initialize variables for tracking the best validation score and patience count.
    best_val = 0
    patience_count = 1
    epoch = 1

    # Continue training until reaching the maximum number of epochs.
    while epoch <= num_epochs:
        # Check if the patience count is within the allowed limit.
        if patience_count <= patience:
            # Set the model to train mode and compute the training loss for the current epoch.
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)

            # Set the model to eval mode and compute the validation score.
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)

            # Check if the current validation score is better than the best validation score so far.
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')

                # Create a dictionary to store the checkpoint information.
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})

                # Save the checkpoint to a file using cloudpickle.
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)

                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            # Print the training and validation scores for the current epoch.
            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(epoch, num_epochs, loss_train, score_val))
            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")

        epoch += 1

    # Save the best model by copying the checkpoint directory.
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    # Print the final results.
    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")


##### Function to compute test set score of the final saved model

##### Train the model and evaluate its performance

In [17]:
def test_evaluate():
    # Create the final model
    final_model = GNN(config, global_size, num_tasks)

    # Set the path to the best model checkpoint file
    path = os.path.join(best_model_path, 'checkpoint.pth')

    # Open the best model checkpoint file and load it
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)

    # Load the state dictionary of the best model
    final_model.load_state_dict(checkpoint["model_state_dict"])

    # Set the final model to evaluation mode
    final_model.eval()

    # Compute the test score
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    # Print the test score
    print("Test Score: {:.3f}".format(test_score), "\n")

    # Print the execution time
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))


In [18]:
#This line imports the time module, which provides various time-related functions
import time
#This line records the current time using time.time() and assigns it to the variable start_time. It serves as the starting point for measuring the execution time
start_time = time.time()
#This line calls the train_evaluate() function, which is likely responsible for training and evaluating a model.
train_evaluate()
#This line calls the test_evaluate() function, which probably performs evaluation on a separate test dataset.
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.641 | Valid Score: 0.254
 
Epoch: 1/100 | Best Valid Score Until Now: 0.254 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.620 | Valid Score: 0.264
 
Epoch: 2/100 | Best Valid Score Until Now: 0.264 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.608 | Valid Score: 0.271
 
Epoch: 3/100 | Best Valid Score Until Now: 0.271 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.601 | Valid Score: 0.281
 
Epoch: 4/100 | Best Valid Score Until Now: 0.281 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.595 | Valid Score: 0.290
 
Epoch: 5/100 | Best Valid Score Until Now: 0.290 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.590 | Valid Score: 0.303
 
Epoch: 6/100 | Best Valid Score Until Now: 0.303 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.588 | Valid Score: 0.319
 
Epoch: 7/100 | Best Valid Score Until Now: 0.319 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.584 | Valid Score: 0.336
 
Epoch: 8/100 | Best Valid Score Until Now: 

#GCN 3Layer


In [40]:
class GNN(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        
        self.node_feature_size = self.config.get('node_feature_size', 127)

        
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

       
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = GraphConv(self.node_feature_size, self.hidden_size,allow_zero_in_degree=True)
        self.conv2 = GraphConv(self.hidden_size, self.hidden_size,allow_zero_in_degree=True)
        self.conv3 = GraphConv(self.hidden_size, self.num_tasks,allow_zero_in_degree=True)

    
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.conv3(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

In [41]:
from sklearn.metrics import roc_auc_score

def compute_score(model, data_loader, val_size, num_tasks):
    model.eval()
    metric = roc_auc_score
    with torch.no_grad():
        prediction_all= torch.empty(0)
        labels_all= torch.empty(0)
        masks_all= torch.empty(0)
        for i, (mol_dgl_graph, labels, masks, globals) in enumerate(data_loader):
            prediction = model(mol_dgl_graph, globals)
            prediction = torch.sigmoid(prediction)
            prediction_all = torch.cat((prediction_all, prediction), 0)
            labels_all = torch.cat((labels_all, labels), 0)
            masks_all = torch.cat((masks_all, masks), 0)
        average = torch.tensor([0.])
        for i in range(num_tasks):
            a1 = prediction_all[:, i][masks_all[:,i]==1]
            a2 = labels_all[:, i][masks_all[:,i]==1]
            try:
                t = metric(a2.int().cpu(), a1.cpu()).item()
            except ValueError:
                t = 0
            average += t
    return average.item()/num_tasks

In [42]:
def loss_func(output, label, mask, num_tasks):
   
    pos_weight = torch.ones((1, num_tasks))
    pos_weight
    criterion = torch.nn.BCEWithLogitsLoss(reduction='none', pos_weight=pos_weight)
    loss = mask*criterion(output,label)
    loss = loss.sum() / mask.sum()
    return loss

In [43]:
def train_epoch(train_dataloader, model, optimizer):
    epoch_train_loss = 0
    iterations = 0
    model.train() 
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(train_dataloader):
        prediction = model(mol_dgl_graph, globals)
        loss_train = loss_func(prediction, labels, masks, num_tasks)
        optimizer.zero_grad(set_to_none=True)
        loss_train.backward()
        optimizer.step()
        epoch_train_loss += loss_train.detach().item()
        iterations += 1
    epoch_train_loss /= iterations
    return epoch_train_loss

In [44]:
def train_evaluate():

    model = GNN(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")


In [45]:
def test_evaluate():
    final_model = GNN(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))


In [46]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.666 | Valid Score: 0.237
 
Epoch: 1/100 | Best Valid Score Until Now: 0.237 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.629 | Valid Score: 0.248
 
Epoch: 2/100 | Best Valid Score Until Now: 0.248 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.611 | Valid Score: 0.259
 
Epoch: 3/100 | Best Valid Score Until Now: 0.259 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.601 | Valid Score: 0.275
 
Epoch: 4/100 | Best Valid Score Until Now: 0.275 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.593 | Valid Score: 0.298
 
Epoch: 5/100 | Best Valid Score Until Now: 0.298 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.584 | Valid Score: 0.328
 
Epoch: 6/100 | Best Valid Score Until Now: 0.328 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.577 | Valid Score: 0.373
 
Epoch: 7/100 | Best Valid Score Until Now: 0.373 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.570 | Valid Score: 0.451
 
Epoch: 8/100 | Best Valid Score Until Now: 

#GraphSAGE 2Layer


In [19]:
class GNN(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        
        self.node_feature_size = self.config.get('node_feature_size', 127)

        
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv(self.node_feature_size, self.hidden_size, aggregator_type='mean')
        self.conv2 = SAGEConv(self.hidden_size, self.num_tasks, aggregator_type='mean')

    
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

In [20]:
from sklearn.metrics import roc_auc_score

def compute_score(model, data_loader, val_size, num_tasks):
    
    model.eval()
    metric = roc_auc_score
    with torch.no_grad():
        
        prediction_all= torch.empty(0)
        labels_all= torch.empty(0)
        masks_all= torch.empty(0)
        
        for i, (mol_dgl_graph, labels, masks, globals) in enumerate(data_loader):
            prediction = model(mol_dgl_graph, globals)
            prediction = torch.sigmoid(prediction)
            prediction_all = torch.cat((prediction_all, prediction), 0)
            labels_all = torch.cat((labels_all, labels), 0)
            masks_all = torch.cat((masks_all, masks), 0)
        average = torch.tensor([0.])
        for i in range(num_tasks):
            a1 = prediction_all[:, i][masks_all[:,i]==1]
            a2 = labels_all[:, i][masks_all[:,i]==1]
            try:
                t = metric(a2.int().cpu(), a1.cpu()).item()
            except ValueError:
                t = 0
            average += t
    return average.item()/num_tasks

In [21]:
def loss_func(output, label, mask, num_tasks):
    pos_weight = torch.ones((1, num_tasks))
    pos_weight
    criterion = torch.nn.BCEWithLogitsLoss(reduction='none', pos_weight=pos_weight)
    loss = mask*criterion(output,label)
    loss = loss.sum() / mask.sum()
    return loss

In [22]:
def train_epoch(train_dataloader, model, optimizer):
    epoch_train_loss = 0
    iterations = 0
    model.train() 
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(train_dataloader):
        prediction = model(mol_dgl_graph, globals)
        loss_train = loss_func(prediction, labels, masks, num_tasks)
        optimizer.zero_grad(set_to_none=True)
        loss_train.backward()
        optimizer.step()
        epoch_train_loss += loss_train.detach().item()
        iterations += 1
    epoch_train_loss /= iterations
    return epoch_train_loss

In [23]:
def train_evaluate():
    model = GNN(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
    best_val = 0
    patience_count = 1
    epoch = 1
    while epoch <= num_epochs:
       
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))
            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)
    
    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")
    

In [24]:
def test_evaluate():
    final_model = GNN(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')

    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)
    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [25]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.538 | Valid Score: 0.795
 
Epoch: 1/100 | Best Valid Score Until Now: 0.795 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.473 | Valid Score: 0.821
 
Epoch: 2/100 | Best Valid Score Until Now: 0.821 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.438 | Valid Score: 0.829
 
Epoch: 3/100 | Best Valid Score Until Now: 0.829 

Patience 1
Epoch: 4/100 | Training Loss: 0.422 | Valid Score: 0.827
 
Epoch: 4/100 | Best Valid Score Until Now: 0.829 

Patience 2
Epoch: 5/100 | Training Loss: 0.421 | Valid Score: 0.824
 
Epoch: 5/100 | Best Valid Score Until Now: 0.829 

Patience 3
Epoch: 6/100 | Training Loss: 0.405 | Valid Score: 0.828
 
Epoch: 6/100 | Best Valid Score Until Now: 0.829 

Patience 4
Epoch: 7/100 | Training Loss: 0.399 | Valid Score: 0.827
 
Epoch: 7/100 | Best Valid Score Until Now: 0.829 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.396 | Valid Score: 0.829
 
Epoch: 8/100 | Best Valid Score Until Now: 0.829 

Patience 1
E

#GraphSAGE 3Layer

In [54]:
class GNN(nn.Module):
    def __init__(self, config, global_size=200, num_tasks=1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv(self.node_feature_size, self.hidden_size,aggregator_type='mean')
        self.conv2 = SAGEConv(self.hidden_size, self.hidden_size,aggregator_type='mean')
        self.conv3 = SAGEConv(self.hidden_size, self.num_tasks,aggregator_type='mean')

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]

        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.conv3(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h

        return dgl.mean_nodes(mol_dgl_graph, "h")


In [57]:
from sklearn.metrics import roc_auc_score

def compute_score(model, data_loader, val_size, num_tasks):
    model.eval()
    metric = roc_auc_score
    with torch.no_grad():
        prediction_all= torch.empty(0)
        labels_all= torch.empty(0)
        masks_all= torch.empty(0)
        for i, (mol_dgl_graph, labels, masks, globals) in enumerate(data_loader):
            prediction = model(mol_dgl_graph, globals)
            prediction = torch.sigmoid(prediction)
            prediction_all = torch.cat((prediction_all, prediction), 0)
            labels_all = torch.cat((labels_all, labels), 0)
            masks_all = torch.cat((masks_all, masks), 0)
        average = torch.tensor([0.])
        for i in range(num_tasks):
            a1 = prediction_all[:, i][masks_all[:,i]==1]
            a2 = labels_all[:, i][masks_all[:,i]==1]
            try:
                t = metric(a2.int().cpu(), a1.cpu()).item()
            except ValueError:
                t = 0
            average += t
    return average.item()/num_tasks

In [58]:
def loss_func(output, label, mask, num_tasks):
    pos_weight = torch.ones((1, num_tasks))
    pos_weight
    criterion = torch.nn.BCEWithLogitsLoss(reduction='none', pos_weight=pos_weight)
    loss = mask*criterion(output,label)
    loss = loss.sum() / mask.sum()
    return loss

In [59]:
def train_epoch(train_dataloader, model, optimizer):
    epoch_train_loss = 0
    iterations = 0
    model.train() 
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(train_dataloader):
        prediction = model(mol_dgl_graph, globals)
        loss_train = loss_func(prediction, labels, masks, num_tasks)
        
        optimizer.zero_grad(set_to_none=True)
        
        loss_train.backward()
        
        optimizer.step()
        
        epoch_train_loss += loss_train.detach().item()
        iterations += 1
    epoch_train_loss /= iterations
    return epoch_train_loss

In [60]:
def train_evaluate():

    model = GNN(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")


In [61]:
def test_evaluate():
    final_model = GNN(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))


In [62]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.642 | Valid Score: 0.413
 
Epoch: 1/100 | Best Valid Score Until Now: 0.413 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.587 | Valid Score: 0.579
 
Epoch: 2/100 | Best Valid Score Until Now: 0.579 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.545 | Valid Score: 0.731
 
Epoch: 3/100 | Best Valid Score Until Now: 0.731 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.518 | Valid Score: 0.809
 
Epoch: 4/100 | Best Valid Score Until Now: 0.809 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.500 | Valid Score: 0.829
 
Epoch: 5/100 | Best Valid Score Until Now: 0.829 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.490 | Valid Score: 0.830
 
Epoch: 6/100 | Best Valid Score Until Now: 0.830 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.475 | Valid Score: 0.837
 
Epoch: 7/100 | Best Valid Score Until Now: 0.837 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.462 | Valid Score: 0.839
 
Epoch: 8/100 | Best Valid Score Until Now: 

#GIN 2Layer


In [77]:
class GNN(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = GINConv(nn.Linear(self.node_feature_size, self.hidden_size), aggregator_type='sum')
        self.conv2 = GINConv(nn.Linear(self.hidden_size, self.num_tasks), aggregator_type='sum')

    
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

In [71]:
from sklearn.metrics import roc_auc_score

def compute_score(model, data_loader, val_size, num_tasks):
  
    model.eval()
    metric = roc_auc_score
    with torch.no_grad():
       
        prediction_all= torch.empty(0)
        labels_all= torch.empty(0)
        masks_all= torch.empty(0)
        
        for i, (mol_dgl_graph, labels, masks, globals) in enumerate(data_loader):
            prediction = model(mol_dgl_graph, globals)
            prediction = torch.sigmoid(prediction)
           
            prediction_all = torch.cat((prediction_all, prediction), 0)
            labels_all = torch.cat((labels_all, labels), 0)
            masks_all = torch.cat((masks_all, masks), 0)
        average = torch.tensor([0.])
        for i in range(num_tasks):
            a1 = prediction_all[:, i][masks_all[:,i]==1]
            a2 = labels_all[:, i][masks_all[:,i]==1]
            try:
                t = metric(a2.int().cpu(), a1.cpu()).item()
            except ValueError:
                t = 0
            average += t
    return average.item()/num_tasks

In [72]:
def loss_func(output, label, mask, num_tasks):
    pos_weight = torch.ones((1, num_tasks))
    pos_weight
    criterion = torch.nn.BCEWithLogitsLoss(reduction='none', pos_weight=pos_weight)
    loss = mask*criterion(output,label)
    loss = loss.sum() / mask.sum()
    return loss

In [73]:
def train_epoch(train_dataloader, model, optimizer):
    epoch_train_loss = 0
    iterations = 0
    model.train() 
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(train_dataloader):
        prediction = model(mol_dgl_graph, globals)
        loss_train = loss_func(prediction, labels, masks, num_tasks)
        optimizer.zero_grad(set_to_none=True)
        loss_train.backward()
        optimizer.step()
        epoch_train_loss += loss_train.detach().item()
        iterations += 1
    epoch_train_loss /= iterations
    return epoch_train_loss

In [74]:
def train_evaluate():

    model = GNN(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")


In [75]:
def test_evaluate():
    final_model = GNN(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [78]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.655 | Valid Score: 0.242
 
Epoch: 1/100 | Best Valid Score Until Now: 0.242 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.622 | Valid Score: 0.273
 
Epoch: 2/100 | Best Valid Score Until Now: 0.273 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.609 | Valid Score: 0.316
 
Epoch: 3/100 | Best Valid Score Until Now: 0.316 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.599 | Valid Score: 0.378
 
Epoch: 4/100 | Best Valid Score Until Now: 0.378 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.588 | Valid Score: 0.444
 
Epoch: 5/100 | Best Valid Score Until Now: 0.444 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.577 | Valid Score: 0.495
 
Epoch: 6/100 | Best Valid Score Until Now: 0.495 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.565 | Valid Score: 0.559
 
Epoch: 7/100 | Best Valid Score Until Now: 0.559 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.557 | Valid Score: 0.617
 
Epoch: 8/100 | Best Valid Score Until Now: 

#GIN 3Layer

In [79]:
class GNN(nn.Module):
    def __init__(self, config, global_size=200, num_tasks=1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = GINConv(nn.Linear(self.node_feature_size, self.hidden_size), aggregator_type='sum')
        self.conv2 = GINConv(nn.Linear(self.hidden_size, self.hidden_size), aggregator_type='sum')
        self.conv3 = GINConv(nn.Linear(self.hidden_size, self.num_tasks), aggregator_type='sum')

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]

        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.conv3(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h

        return dgl.mean_nodes(mol_dgl_graph, "h")


In [80]:
from sklearn.metrics import roc_auc_score

def compute_score(model, data_loader, val_size, num_tasks):
    model.eval()
    metric = roc_auc_score
    with torch.no_grad():
        prediction_all= torch.empty(0)
        labels_all= torch.empty(0)
        masks_all= torch.empty(0)
        for i, (mol_dgl_graph, labels, masks, globals) in enumerate(data_loader):
            prediction = model(mol_dgl_graph, globals)
            prediction = torch.sigmoid(prediction)
            prediction_all = torch.cat((prediction_all, prediction), 0)
            labels_all = torch.cat((labels_all, labels), 0)
            masks_all = torch.cat((masks_all, masks), 0)
        average = torch.tensor([0.])
        for i in range(num_tasks):
            a1 = prediction_all[:, i][masks_all[:,i]==1]
            a2 = labels_all[:, i][masks_all[:,i]==1]
            try:
                t = metric(a2.int().cpu(), a1.cpu()).item()
            except ValueError:
                t = 0
            average += t
    return average.item()/num_tasks

In [81]:
def loss_func(output, label, mask, num_tasks):
    pos_weight = torch.ones((1, num_tasks))
    pos_weight
    criterion = torch.nn.BCEWithLogitsLoss(reduction='none', pos_weight=pos_weight)
    loss = mask*criterion(output,label)
    loss = loss.sum() / mask.sum()
    return loss

In [82]:
def train_epoch(train_dataloader, model, optimizer):
    epoch_train_loss = 0
    iterations = 0
    model.train() 
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(train_dataloader):
        prediction = model(mol_dgl_graph, globals)
        loss_train = loss_func(prediction, labels, masks, num_tasks)
        optimizer.zero_grad(set_to_none=True)
        loss_train.backward()
        optimizer.step()
        epoch_train_loss += loss_train.detach().item()
        iterations += 1
    epoch_train_loss /= iterations
    return epoch_train_loss

In [83]:
def train_evaluate():

    model = GNN(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")


In [84]:
def test_evaluate():
    final_model = GNN(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))


In [85]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.607 | Valid Score: 0.483
 
Epoch: 1/100 | Best Valid Score Until Now: 0.483 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.573 | Valid Score: 0.574
 
Epoch: 2/100 | Best Valid Score Until Now: 0.574 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.548 | Valid Score: 0.747
 
Epoch: 3/100 | Best Valid Score Until Now: 0.747 

Patience 1
Epoch: 4/100 | Training Loss: 0.526 | Valid Score: 0.738
 
Epoch: 4/100 | Best Valid Score Until Now: 0.747 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.506 | Valid Score: 0.784
 
Epoch: 5/100 | Best Valid Score Until Now: 0.784 

Patience 1
Epoch: 6/100 | Training Loss: 0.492 | Valid Score: 0.772
 
Epoch: 6/100 | Best Valid Score Until Now: 0.784 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.481 | Valid Score: 0.813
 
Epoch: 7/100 | Best Valid Score Until Now: 0.813 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.470 | Valid Score: 0.819
 
Epoch: 8/100 | Best Valid Score Until Now: 0.819 

Sa

#GAT 2Layer

In [86]:
class GNN(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.num_heads = self.config.get('num_heads', 1)
        self.dropout = self.config.get('dropout', 0.0)

        self.conv1 = GATConv(
            self.node_feature_size,
            self.hidden_size,
            num_heads=self.num_heads,
            feat_drop=self.dropout,
            attn_drop=self.dropout,allow_zero_in_degree=True)

        self.fc = nn.Linear(
            self.hidden_size * self.num_heads,
            self.hidden_size)
        self.conv2 = GATConv(
            self.hidden_size,
            self.num_tasks,
            num_heads=1,
            feat_drop=self.dropout,
            attn_drop=self.dropout,allow_zero_in_degree=True
       )
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]

        # First GAT layer
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"]).flatten(1)
        h = F.relu(h)
        h = self.fc(h)
        h = F.dropout(h, p=self.dropout, training=self.training)

        # Second GAT layer
        h = self.conv2(mol_dgl_graph, h).squeeze(1)
        mol_dgl_graph.ndata["h"] = h

        return dgl.mean_nodes(mol_dgl_graph, "h")


In [87]:
import math
from sklearn.metrics import mean_squared_error
def compute_score(model, data_loader, val_size, num_tasks, scaler=None):
  model.eval()
  loss_sum = nn.MSELoss(reduction='sum') # MSE with sum instead of mean, i.e., sum_i[(y_i)^2-(y'_i)^2]
  final_loss = 0
  state = torch.get_rng_state()
  with torch.no_grad():
            for i, (mol_dgl_graph, labels, masks, globals) in enumerate(data_loader):
                prediction = model(mol_dgl_graph, globals)
                if scaler is not None:
                  prediction = torch.tensor(scaler.inverse_transform(prediction.detach().cpu()))
                  labels = torch.tensor(scaler.inverse_transform(labels.cpu()))
                loss = loss_sum(prediction, labels)
                final_loss += loss.item()
            final_loss /= val_size
            final_loss = math.sqrt(final_loss) # RMSE
  return final_loss / num_tasks

In [88]:
def loss_func(output, label, mask, num_tasks):
    pos_weight = torch.ones((1, num_tasks))
    pos_weight
    criterion = nn.MSELoss(reduction='none')
    loss = mask*criterion(output,label)
    loss = loss.sum() / mask.sum()
    return loss

In [89]:
def train_epoch(train_dataloader, model, optimizer):
    epoch_train_loss = 0
    iterations = 0
    model.train() 
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(train_dataloader):
        prediction = model(mol_dgl_graph, globals)
        loss_train = loss_func(prediction, labels, masks, num_tasks)
        optimizer.zero_grad(set_to_none=True)
        loss_train.backward()
        optimizer.step()
        epoch_train_loss += loss_train.detach().item()
        iterations += 1
    epoch_train_loss /= iterations
    return epoch_train_loss

In [90]:
def train_evaluate():

    model = GNN(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")


In [91]:
def test_evaluate():
    final_model = GNN(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [92]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.621 | Valid Score: 0.535
 
Epoch: 1/100 | Best Valid Score Until Now: 0.535 

Patience 1
Epoch: 2/100 | Training Loss: 0.307 | Valid Score: 0.512
 
Epoch: 2/100 | Best Valid Score Until Now: 0.535 

Patience 2
Epoch: 3/100 | Training Loss: 0.281 | Valid Score: 0.505
 
Epoch: 3/100 | Best Valid Score Until Now: 0.535 

Patience 3
Epoch: 4/100 | Training Loss: 0.265 | Valid Score: 0.485
 
Epoch: 4/100 | Best Valid Score Until Now: 0.535 

Patience 4
Epoch: 5/100 | Training Loss: 0.249 | Valid Score: 0.468
 
Epoch: 5/100 | Best Valid Score Until Now: 0.535 

Patience 5
Epoch: 6/100 | Training Loss: 0.236 | Valid Score: 0.452
 
Epoch: 6/100 | Best Valid Score Until Now: 0.535 

Patience 6
Epoch: 7/100 | Training Loss: 0.223 | Valid Score: 0.439
 
Epoch: 7/100 | Best Valid Score Until Now: 0.535 

Patience 7
Epoch: 8/100 | Training Loss: 0.212 | Valid Score: 0.424
 
Epoch: 8/100 | Best Valid Score Until Now: 0.535 

Patience 8
Epoch: 9/100 | T

#GAT 3Layer

In [93]:
class GNN(nn.Module):
    def __init__(self, config, global_size=200, num_tasks=1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)


        # Number of attention heads
        self.num_heads = self.config.get('num_heads', 1)

        
        self.dropout = self.config.get('dropout', 0.0)

        # GAT layer1
        self.conv1 = GATConv(self.node_feature_size,self.hidden_size,num_heads=self.num_heads,feat_drop=self.dropout,attn_drop=self.dropout,allow_zero_in_degree=True)

        # Linear layer
        self.fc = nn.Linear(self.hidden_size * self.num_heads,self.hidden_size)

        # GAT layer2
        self.conv2 = GATConv(self.hidden_size,self.hidden_size,num_heads=1,feat_drop=self.dropout,attn_drop=self.dropout,allow_zero_in_degree=True)

        # GAT layer3
        self.conv3 = GATConv(self.hidden_size,self.num_tasks,num_heads=1,feat_drop=self.dropout,attn_drop=self.dropout,allow_zero_in_degree=True)

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]

        # First GAT layer1
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"]).flatten(1)
        h = F.relu(h)
        h = self.fc(h)
        h = F.dropout(h, p=self.dropout, training=self.training)

        # Second GAT layer2
        h = self.conv2(mol_dgl_graph, h).squeeze(1)
        h = F.relu(h)
        h = F.dropout(h, p=self.dropout, training=self.training)

        # Third GAT layer3
        if self.num_tasks == 1:
            h = self.conv3(mol_dgl_graph, h).squeeze(1)
        else:
            hs = []
            for i in range(self.num_tasks):
                hi = self.conv3(mol_dgl_graph, h).squeeze(1)
                hs.append(hi)
            h = torch.stack(hs, dim=1)

        mol_dgl_graph.ndata["h"] = h

        if self.num_tasks == 1:
            return dgl.mean_nodes(mol_dgl_graph, "h")
        else:
            return dgl.mean_nodes(mol_dgl_graph, "h"), h


In [94]:
import math
from sklearn.metrics import mean_squared_error
def compute_score(model, data_loader, val_size, num_tasks, scaler=None):
  model.eval()
  loss_sum = nn.MSELoss(reduction='sum') # MSE with sum instead of mean, i.e., sum_i[(y_i)^2-(y'_i)^2]
  final_loss = 0
  state = torch.get_rng_state()
  with torch.no_grad():
            for i, (mol_dgl_graph, labels, masks, globals) in enumerate(data_loader):
                prediction = model(mol_dgl_graph, globals)
                if scaler is not None:
                  prediction = torch.tensor(scaler.inverse_transform(prediction.detach().cpu()))
                  labels = torch.tensor(scaler.inverse_transform(labels.cpu()))
                loss = loss_sum(prediction, labels)
                final_loss += loss.item()
            final_loss /= val_size
            final_loss = math.sqrt(final_loss) # RMSE
  return final_loss / num_tasks

In [95]:
def loss_func(output, label, mask, num_tasks):
    pos_weight = torch.ones((1, num_tasks))
    pos_weight
    criterion = nn.MSELoss(reduction='none')
    loss = mask*criterion(output,label)
    loss = loss.sum() / mask.sum()
    return loss

In [96]:
def train_epoch(train_dataloader, model, optimizer):
    epoch_train_loss = 0
    iterations = 0
    model.train() 
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(train_dataloader):
        prediction = model(mol_dgl_graph, globals)
        loss_train = loss_func(prediction, labels, masks, num_tasks)
        optimizer.zero_grad(set_to_none=True)
        loss_train.backward()
        optimizer.step()
        epoch_train_loss += loss_train.detach().item()
        iterations += 1
    epoch_train_loss /= iterations
    return epoch_train_loss

In [97]:
def train_evaluate():

    model = GNN(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")


In [98]:
def test_evaluate():
    final_model = GNN(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [99]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.352 | Valid Score: 0.464
 
Epoch: 1/100 | Best Valid Score Until Now: 0.464 

Patience 1
Epoch: 2/100 | Training Loss: 0.244 | Valid Score: 0.447
 
Epoch: 2/100 | Best Valid Score Until Now: 0.464 

Patience 2
Epoch: 3/100 | Training Loss: 0.210 | Valid Score: 0.412
 
Epoch: 3/100 | Best Valid Score Until Now: 0.464 

Patience 3
Epoch: 4/100 | Training Loss: 0.191 | Valid Score: 0.389
 
Epoch: 4/100 | Best Valid Score Until Now: 0.464 

Patience 4
Epoch: 5/100 | Training Loss: 0.182 | Valid Score: 0.386
 
Epoch: 5/100 | Best Valid Score Until Now: 0.464 

Patience 5
Epoch: 6/100 | Training Loss: 0.173 | Valid Score: 0.370
 
Epoch: 6/100 | Best Valid Score Until Now: 0.464 

Patience 6
Epoch: 7/100 | Training Loss: 0.165 | Valid Score: 0.359
 
Epoch: 7/100 | Best Valid Score Until Now: 0.464 

Patience 7
Epoch: 8/100 | Training Loss: 0.159 | Valid Score: 0.350
 
Epoch: 8/100 | Best Valid Score Until Now: 0.464 

Patience 8
Epoch: 9/100 | T